# Important data

In [29]:
import pandas as pd
import numpy as np
import mlflow

from sklearn import datasets
from sklearn.model_selection import train_test_split


from evidently.dashboard import Dashboard
from evidently.tabs import NumTargetDriftTab,CatTargetDriftTab

# Target Drift

In [30]:
reference_data = pd.read_csv("training_data.csv",
                                            header=None,
                                            names=[ "day{}".format(i) for i in range(0,14) ]+["target"] )


In [31]:
production_scored_data = pd.read_csv("scored_data.csv",
                                            header=None,
                                            names=[ "day{}".format(i) for i in range(0,14) ]+["target"] )


In [32]:
production_scored_data

,day0,day1,day2,day3,day4,day5,day6,day7,day8,day9,day10,day11,day12,day13,target
0,1,1,0,0,0,0,1,1,0,0,0,1,0,0,1
1,1,0,0,0,0,1,1,0,0,0,1,0,0,0,1
2,0,0,0,0,1,1,0,0,0,1,0,0,0,1,0
3,0,0,0,1,1,0,0,0,1,0,0,0,1,0,1
4,0,0,1,1,0,0,0,1,0,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,0,0,0,1,0,0,1,1,1,0,1,0,1,1,0
73,0,0,1,0,0,1,1,1,0,1,0,1,1,0,1
74,0,1,0,0,1,1,1,0,1,0,1,1,0,1,0
75,1,0,0,1,1,1,0,1,0,1,1,0,1,1,0


In [33]:
EXPERIMENT_NAME="./reports_target_drift"
mlflow.set_experiment(EXPERIMENT_NAME)
with mlflow.start_run():
    model_target_drift = Dashboard(tabs=[CatTargetDriftTab])
    model_target_drift.calculate(reference_data,production_scored_data)
    model_target_drift.save(EXPERIMENT_NAME+"/target_drift.html")
    model_target_drift._save_to_json(EXPERIMENT_NAME+"/target_drift.json")
    mlflow.log_artifacts(EXPERIMENT_NAME)